In [1]:
import os
import sys

import yaml
import pymysql
import pandas as pd
import pytablewriter
from IPython.display import clear_output

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%load_ext sql

sys.path.insert(0, '..')

with open('../config.yml') as f:
    config = yaml.load(f)

DB = 'mysql+pymysql://{username}:{password}@{host}/{database}?charset=utf8'.format(
    **config['develop']
)

WRITER = pytablewriter.MarkdownTableWriter()

pd.options.display.max_rows = 10

clear_output()

In [2]:
def query(q):
    df = pd.read_sql_query(q, DB)
    WRITER.from_dataframe(df)
    WRITER.write_table()

- pandas.DataFrameをMarkdownのテーブル記法に変換する方法（[4.5.3. Using pandas DataFrame as tabular data source](http://pytablewriter.readthedocs.io/en/latest/pages/examples/datasource/from_pandas_dataframe.html)）
- 参考文献
    - L. Beighley,直生佐藤,多苗子松永,Head first SQL : 頭とからだで覚える SQL の基本,オライリー・ジャパン, オーム社 (発売),2008.

# 各テーブルの内容

## corporaテーブル

In [3]:
query(
    'select * from corpora limit 20',
)

|id |corpus_id|                     name                     |
|--:|---------|----------------------------------------------|
|  1|CPRTUR   |楽天データセット::楽天トラベル::ユーザレビュー|



## corpus_filesテーブル

In [5]:
query(
    'select * from corpus_files limit 20',
)

|id |          corpus_file_id          |corpus_id|                            checksum                            |
|--:|----------------------------------|---------|----------------------------------------------------------------|
|  2|travel02_userReview00_20160304.txt|CPRTUR   |729a3fae33cb900ed5fb56dffd2e6418790646260cc3b71326df42d160cc1a97|



## corpus_datumテーブル

In [9]:
query(
    'select * from corpus_datum limit 5',
)

|id |corpus_data_id|          corpus_file_id          |id_in_corpus|                        text                        |
|--:|--------------|----------------------------------|-----------:|----------------------------------------------------|
|101|CPRTUR00000001|travel02_userReview00_20160304.txt|           1|このホテルでこの値段は安い。                        |
|102|CPRTUR00000008|travel02_userReview00_20160304.txt|           8|駅から出てすぐの所にありました。雨でも安心ですね    |
|103|CPRTUR00000009|travel02_userReview00_20160304.txt|           9|乗車券が有効利用できました                          |
|104|CPRTUR00000012|travel02_userReview00_20160304.txt|          12|バスルームが広いのがよかったです。                  |
|105|CPRTUR00000022|travel02_userReview00_20160304.txt|          22|ラウンジでコーヒーを頼んだらプチケーキが付いてきた！|



In [10]:
query('''
    
SELECT
    *
FROM
    corpus_datum as cd
ORDER BY
    cd.text
LIMIT 1
    
''')

|id |corpus_data_id|          corpus_file_id          |id_in_corpus|                                                                                                                                  text                                                                                                                                  |
|--:|--------------|----------------------------------|-----------:|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|185|CPRTUR00000346|travel02_userReview00_20160304.txt|         346|8月14日一泊しました。TDLに行くには駅前で便利だと思います。最後の花火を見た後、車で、35分ぐらいで着いたと思います。ホテルの近くに、薬局も遅くまで営業していて、助かりました。室内に冷蔵庫はついていませんでしたが、ビールは廊下に自販機がありました。清潔なホテルでした。|



## sentence_delimitersテーブル

In [11]:
query(
    'select * from sentence_delimiters limit 5',
)

|id |sentence_delimiter_id|      regex       |
|--:|---------------------|------------------|
|  1|SD0001               |[。．\.！!？\?\n]+|



## sentencesテーブル

In [12]:
query(
    'select * from sentences limit 5',
)

|id |corpus_data_id|sentence_delimiter_id|       sentence_id        |              text              |nth|length|
|--:|--------------|---------------------|--------------------------|--------------------------------|--:|-----:|
|  1|CPRTUR00000001|SD0001               |CPRTUR00000001_SD0001_0101|このホテルでこの値段は安い      |  1|     1|
|  2|CPRTUR00000008|SD0001               |CPRTUR00000008_SD0001_0201|駅から出てすぐの所にありました  |  1|     2|
|  3|CPRTUR00000008|SD0001               |CPRTUR00000008_SD0001_0202|雨でも安心ですね                |  2|     2|
|  4|CPRTUR00000009|SD0001               |CPRTUR00000009_SD0001_0101|乗車券が有効利用できました      |  1|     1|
|  5|CPRTUR00000012|SD0001               |CPRTUR00000012_SD0001_0101|バスルームが広いのがよかったです|  1|     1|



## morphological_analysiesテーブル

In [13]:
query(
    'select * from morphological_analysies limit 5',
)

|id |  morphological_analysies_id   |       sentence_id        |surface| pos  | pos1 |pos2|pos3|ctype|cform| base | yomi | pron |nth|length|
|--:|-------------------------------|--------------------------|-------|------|------|----|----|-----|-----|------|------|------|--:|-----:|
|  1|CPRTUR00000001_SD0001_0101_0701|CPRTUR00000001_SD0001_0101|この   |連体詞|      |    |    |     |     |この  |コノ  |コノ  |  1|     7|
|  2|CPRTUR00000001_SD0001_0101_0702|CPRTUR00000001_SD0001_0101|ホテル |名詞  |一般  |    |    |     |     |ホテル|ホテル|ホテル|  2|     7|
|  3|CPRTUR00000001_SD0001_0101_0703|CPRTUR00000001_SD0001_0101|で     |助詞  |格助詞|一般|    |     |     |で    |デ    |デ    |  3|     7|
|  4|CPRTUR00000001_SD0001_0101_0704|CPRTUR00000001_SD0001_0101|この   |連体詞|      |    |    |     |     |この  |コノ  |コノ  |  4|     7|
|  5|CPRTUR00000001_SD0001_0101_0705|CPRTUR00000001_SD0001_0101|値段   |名詞  |一般  |    |    |     |     |値段  |ネダン|ネダン|  5|     7|



## morphemeテーブル

In [15]:
query(
    'select * from morphemes limit 5',
)

|id |morpheme_id|surface| pos  |  pos1  |pos2|pos3|  ctype   |cform |base|yomi|pron|
|--:|-----------|-------|------|--------|----|----|----------|------|----|----|----|
|  1|MO00000001 |JR     |名詞  |固有名詞|組織|    |          |      |    |    |    |
|  2|MO00000002 |、     |記号  |読点    |    |    |          |      |、  |、  |、  |
|  3|MO00000003 |あり   |動詞  |自立    |    |    |ラ変      |基本形|あり|アリ|アリ|
|  4|MO00000004 |あり   |動詞  |自立    |    |    |五段・ラ行|連用形|ある|アリ|アリ|
|  5|MO00000005 |お     |接頭詞|名詞接続|    |    |          |      |お  |オ  |オ  |

